# Imports

In [181]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  StandardScaler
from sksurv.metrics import concordance_index_censored as concordance

In [182]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout,  concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop,SGD,Nadam, Adagrad, Adadelta
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.initializers import Constant ,Orthogonal, RandomNormal, VarianceScaling
from tensorflow.keras.constraints import Constraint, UnitNorm

In [184]:
import timeit
import csv

# Data Preprocessing

In [186]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.max_seq_items = 2000

In [188]:
df_exp = pd.read_csv('pancan_rna.tsv',sep='\t')

In [193]:
df_exp['Hugo_Symbol'] = df_exp['gene_id'].str.split('|',expand=True)[0]

In [194]:
df_exp.index=df_exp['Hugo_Symbol']

In [195]:
df_exp.drop(['Hugo_Symbol','gene_id'],axis=1,inplace=True)

In [196]:
df_exp = df_exp.transpose()

In [198]:
df_exp = df_exp.iloc[:,29:]
df_exp.head()

Hugo_Symbol,A1BG,A1CF,A2BP1,A2LD1,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,AACSL,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAGAB,AAK1,AAMP,AANAT,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA10,ABCA1,ABCA11P,ABCA12,ABCA13,ABCA17P,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB10,ABCB11,ABCB1,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC10,ABCC11,ABCC12,ABCC13,ABCC1,ABCC2,ABCC3,ABCC4,ABCC5,ABCC6,ABCC6P1,ABCC6P2,ABCC8,ABCC9,ABCD1,ABCD2,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABCG2,ABCG4,ABCG5,ABCG8,ABHD10,ABHD11,ABHD12,ABHD12B,ABHD13,ABHD14A,ABHD14B,ABHD15,ABHD1,ABHD2,ABHD3,ABHD4,ABHD5,ABHD6,ABHD8,ABI1,ABI2,ABI3,ABI3BP,ABL1,ABL2,ABLIM1,ABLIM2,ABLIM3,ABO,ABP1,ABR,ABRA,ABT1,ABTB1,ABTB2,ACAA1,ACAA2,ACACA,ACACB,ACAD10,ACAD11,ACAD8,ACAD9,ACADL,ACADM,ACADS,ACADSB,ACADVL,ACAN,ACAP1,ACAP2,ACAP3,ACAT1,ACAT2,ACBD3,ACBD4,ACBD5,ACBD6,ACBD7,ACCN1,ACCN2,ACCN3,ACCN4,ACCN5,ACCS,ACCSL,ACD,ACE,ACE2,ACER1,ACER2,ACER3,ACHE,ACIN1,ACLY,ACMSD,ACN9,ACO1,ACO2,ACOT11,ACOT12,ACOT13,ACOT1,ACOT2,ACOT4,ACOT6,ACOT7,ACOT8,ACOT9,ACOX1,ACOX2,ACOX3,ACOXL,ACP1,ACP2,ACP5,ACP6,ACPL2,ACPP,ACPT,ACR,ACRBP,ACRC,ACRV1,ACSBG1,ACSBG2,ACSF2,ACSF3,ACSL1,ACSL3,ACSL4,ACSL5,ACSL6,ACSM1,ACSM2A,ACSM2B,ACSM3,ACSM4,ACSM5,ACSS1,ACSS2,ACSS3,ACTA1,ACTA2,ACTB,ACTBL2,ACTC1,ACTG1,ACTG2,ACTL6A,ACTL6B,ACTL7A,ACTL7B,ACTL8,ACTL9,ACTN1,ACTN2,ACTN3,ACTN4,ACTR10,ACTR1A,ACTR1B,ACTR2,ACTR3,ACTR3B,ACTR3C,ACTR5,ACTR6,ACTR8,ACTRT1,ACTRT2,ACVR1,ACVR1B,ACVR1C,ACVR2A,ACVR2B,ACVRL1,ACY1,ACY3,ACYP1,ACYP2,ADA,ADAD1,ADAD2,ADAL,ADAM10,ADAM11,ADAM12,ADAM15,ADAM17,ADAM18,ADAM19,ADAM20,ADAM21,...,ZNF594,ZNF595,ZNF596,ZNF597,ZNF598,ZNF599,ZNF600,ZNF605,ZNF606,ZNF607,ZNF608,ZNF609,ZNF610,ZNF611,ZNF613,ZNF614,ZNF615,ZNF616,ZNF618,ZNF619,ZNF620,ZNF621,ZNF622,ZNF623,ZNF624,ZNF625,ZNF626,ZNF627,ZNF628,ZNF629,ZNF630,ZNF638,ZNF639,ZNF641,ZNF642,ZNF643,ZNF644,ZNF645,ZNF646,ZNF648,ZNF649,ZNF652,ZNF653,ZNF654,ZNF655,ZNF658,ZNF660,ZNF662,ZNF664,ZNF665,ZNF667,ZNF668,ZNF669,ZNF670,ZNF671,ZNF672,ZNF673,ZNF674,ZNF675,ZNF676,ZNF677,ZNF678,ZNF679,ZNF680,ZNF681,ZNF682,ZNF683,ZNF684,ZNF687,ZNF688,ZNF689,ZNF691,ZNF692,ZNF695,ZNF696,ZNF69,ZNF697,ZNF699,ZNF700,ZNF701,ZNF702P,ZNF703,ZNF704,ZNF705A,ZNF705D,ZNF706,ZNF707,ZNF70,ZNF708,ZNF709,ZNF710,ZNF711,ZNF713,ZNF714,ZNF71,ZNF716,ZNF717,ZNF718,ZNF720,ZNF721,ZNF727,ZNF732,ZNF735,ZNF737,ZNF738,ZNF740,ZNF746,ZNF74,ZNF747,ZNF749,ZNF750,ZNF75A,ZNF75D,ZNF761,ZNF763,ZNF764,ZNF765,ZNF766,ZNF76,ZNF767,ZNF768,ZNF770,ZNF771,ZNF772,ZNF773,ZNF774,ZNF775,ZNF7,ZNF77,ZNF776,ZNF777,ZNF778,ZNF780A,ZNF780B,ZNF781,ZNF782,ZNF784,ZNF785,ZNF786,ZNF787,ZNF788,ZNF789,ZNF790,ZNF791,ZNF792,ZNF793,ZNF79,ZNF799,ZNF800,ZNF804A,ZNF804B,ZNF805,ZNF80,ZNF808,ZNF813,ZNF81,ZNF814,ZNF815,ZNF816A,ZNF821,ZNF823,ZNF826,ZNF827,ZNF828,ZNF829,ZNF830,ZNF831,ZNF833,ZNF83,ZNF835,ZNF836,ZNF837,ZNF839,ZNF841,ZNF843,ZNF844,ZNF845,ZNF846,ZNF84,ZNF853,ZNF85,ZNF860,ZNF862,ZNF8,ZNF876P,ZNF878,ZNF879,ZNF880,ZNF883,ZNF90,ZNF91,ZNF92,ZNF93,ZNF98,ZNF99,ZNFX1,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZNRF3,ZNRF4,ZP1,ZP2,ZP3,ZP4,ZPBP,ZPBP2,ZPLD1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN10,ZSCAN1,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN18,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN2,ZSCAN29,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-OR-A5J1-01A-11R-A29S-07,16.3305,0.0000,17.2911,182.3920,10373.70,54.7550,190.6820,0.0000,0.0,2225.26,1266.090,0.0000,0.0000,0.0000,0.0,0.0,121.0370,1327.090,868.876,2636.41,1.4409,3520.65,299.712,904.419,101.345,1467.340,946.206,1256.480,27.8578,1580.690,70.6052,319.404,46.2488,0.0000,0.4803,42.2671,804.515,729.587,2.4015,2036.980,14.4092,431.316,4.3228,36.0231,413.064,0.4803,1052.350,363.5930,7.2046,1821.480,199.328,917.387,185.913,808.357,3.3622,0.0,0.0,930.355,3.8425,679.635,173.3910,1212.780,12.0077,1.4409,43.2277,0.0000,17.7714,2158.02,134.9660,719.020,717.579,1375.600,1239.670,978.866,8834.77,346.3020,73.0067,1.4409,3.3622,0.0000,1609.510,541.787,1923.150,0.0000,354.467,507.685,1082.13,247.839,3.3622,2242.07,143.132,1250.240

In [199]:
cols = list(df)
cols[0] = 'Cell_Type'
df.columns=cols
df.index=df['Cell_Type']
df.drop(columns='Cell_Type',inplace=True)
df = df.transpose()

In [200]:
df2 =pd.read_csv('pancan_clinical.tsv',sep='\t',encoding = "ISO-8859-1",low_memory=False)

In [201]:
df2 = df2.iloc[:,:60]

In [202]:
df2.index=df2['bcr_patient_barcode']

In [204]:
df2['vital_status'] = (df2['vital_status']=='Dead').astype('int')

In [205]:
df2.loc[df2['days_to_death']=='[Not Applicable]','days_to_death'] = 0
df2.loc[df2['days_to_death']=='[Not Available]','days_to_death'] = 0
df2.loc[df2['days_to_death']=='[Discrepancy]','days_to_death'] = 0
df2.loc[df2['days_to_death']=='[Completed]','days_to_death'] = 0
df2.loc[df2['days_to_last_followup']=='[Not Available]','days_to_last_followup'] = 0
df2.loc[df2['days_to_last_followup']=='[Not Applicable]','days_to_last_followup'] = 0
df2.loc[df2['days_to_last_followup']=='[Discrepancy]','days_to_last_followup'] = 0
df2.loc[df2['days_to_last_followup']=='[Completed]','days_to_last_followup'] = 0

In [206]:
df2['days_to_last_followup'] = df2['days_to_last_followup'].astype('float32')
df2['days_to_death'] = df2['days_to_death'].astype('float32')

In [207]:
df2['time'] = df2['days_to_death']+df2['days_to_last_followup']

In [208]:
df2.drop(columns=['days_to_death','days_to_last_followup'],inplace=True)

In [209]:
y =df2[['vital_status','time']]

In [211]:
cancer = pd.get_dummies(df2['acronym'])

In [217]:
df_exp.index = df_exp.index.str.replace('-','.')

In [218]:
df_exp.index = df_exp.index.str.slice(0,12)

In [219]:
df.index = df.index.str.slice(0,12)

In [220]:
cancer.index = cancer.index.str.replace('-','.')

In [221]:
y.index = y.index.str.replace('-','.')

In [222]:
df_exp = df_exp.loc[~df_exp.index.duplicated(keep='first')]

In [224]:
y = y[y['time']>0]

In [225]:
df2.index = df2.index.str.replace('-','.')

In [226]:
df = df[df.index.isin(df.index) &df.index.isin(df_exp.index)&df.index.isin(cancer.index)&df.index.isin(y.index)]
df2 = df2[df2.index.isin(df.index) &df2.index.isin(df_exp.index)&df2.index.isin(cancer.index)&df2.index.isin(y.index)]
y = y[y.index.isin(df.index) &y.index.isin(df_exp.index)&y.index.isin(cancer.index)&y.index.isin(y.index)]
cancer = cancer[cancer.index.isin(df.index) &cancer.index.isin(df_exp.index)&cancer.index.isin(cancer.index)&cancer.index.isin(y.index)]
df_exp = df_exp[df_exp.index.isin(df.index) &df_exp.index.isin(df_exp.index)&df_exp.index.isin(cancer.index)&df_exp.index.isin(y.index)]

In [228]:
newindex = y['time'].sort_values().index

In [229]:
y = y.reindex(newindex)
df = df.reindex(newindex)
df_exp = df_exp.reindex(newindex)
cancer = cancer.reindex(newindex)
df2  = df2.reindex(newindex)

In [238]:
df = df.fillna(0)
df_exp = df_exp.fillna(0)
cancer = cancer.fillna(0)
y = y.fillna(0)

In [239]:
SS = StandardScaler()
df = pd.DataFrame(SS.fit_transform(df),index=df.index,columns=df.columns)
X_exp = pd.DataFrame(StandardScaler().fit_transform(df_exp),index=df_exp.index, columns=df_exp.columns)

In [240]:
itm = np.triu(np.ones((len(df),len(df))),0)

In [241]:
itm = pd.DataFrame(itm,index=newindex,columns=newindex)

In [246]:
N = df2[['pathologic_N','clinical_N']]

In [247]:
T = df2[['pathologic_T','clinical_T']]

In [248]:
M = df2[['pathologic_M','clinical_M']]

In [249]:
N = N.replace('[Not Applicable]',0)
N = N.replace('[Not Available]',0)
N = N.replace('[Discrepancy]',0)
N = N.fillna(0)
N = N.astype(str).apply(lambda x: x.str[:2])

In [250]:
T = T.replace('[Not Applicable]',0)
T = T.replace('[Not Available]',0)
T = T.replace('[Discrepancy]',0)
T = T.replace('[Unknown]',0)
T = T.fillna(0)
T = T.astype(str).apply(lambda x: x.str[:2])

In [251]:
M = M.replace('[Not Applicable]',0)
M = M.replace('[Not Available]',0)
M = M.replace('[Discrepancy]',0)
M = M.replace('[Unknown]',0)
M = M.fillna(0)
M = M.astype(str).apply(lambda x: x.str[:2])

In [252]:
N['Stage'] = 0
M['Stage'] = 0
T['Stage'] = 0

In [253]:
for a in N.index:
    if (N.loc[a,'pathologic_N']!=0):
        N.loc[a,'Stage'] = N.loc[a,'pathologic_N']
    elif (N.loc[a,'clinical_N']!=0):
        N.loc[a,'Stage'] = N.loc[a,'clinical_N']
N = N['Stage']   

In [254]:
for a in M.index:
    if (M.loc[a,'pathologic_M']!=0):
        M.loc[a,'Stage'] = M.loc[a,'pathologic_M']
    elif (M.loc[a,'clinical_M']!=0):
        M.loc[a,'Stage'] = M.loc[a,'clinical_M']
M = M['Stage']

In [255]:
for a in T.index:
    if (T.loc[a,'pathologic_T']!=0):
        T.loc[a,'Stage'] = T.loc[a,'pathologic_T']
    elif (T.loc[a,'clinical_T']!=0):
        T.loc[a,'Stage'] = T.loc[a,'clinical_T']
T = T['Stage']

In [256]:
M  = pd.get_dummies(M).astype('bool')
N  = pd.get_dummies(N).astype('bool')
T  = pd.get_dummies(T).astype('bool')

In [257]:
M['0'] = ~M['0']
N['0'] = ~N['0']
T['0'] = ~T['0']

In [258]:
yc = np.array(y)
yc = np.core.records.fromarrays(yc.transpose(),formats='bool,f8')

# Side channel model class

In [110]:
#This class builds the linear cox model with categorical side channels. 
#The categorical side channels are censored so that samples with missing categorical data
#will still impact the cox loss
#This can be easily modified for regression side channels and can be made non-linear by the 
#addition of extra layers or changing the activation functions.

#The class needs to be initialised with data.

class coxnetlin():
    def __init__(self,X,state,time,cancer,itm,m,n,t):
        
        self.m = np.array(m.iloc[:,1:])
        self.mc = np.array(m.iloc[:,0])
        self.n = np.array(n.iloc[:,1:])
        self.nc = np.array(n.iloc[:,0])
        self.t = np.array(t.iloc[:,1:])
        self.tc = np.array(t.iloc[:,0])
        self.X = np.array(X)
        self.state = np.array(state)
        self.time = np.array(time)
        self.cancer = np.array(cancer)
        self.itm = np.array(itm)
        self.itshp=self.itm.shape
      
    #This is the keras custom loss function for cox loss
    def coxloss2(self,ITM, state):
        def loss(y_true, y_pred):
            return -K.mean((y_pred - K.log(K.cumsum(K.exp(K.reverse(y_pred,axes=0))))) * state)

        return loss

    #Custom keras loss function for censored categorical crossentropy    
    def censored_cat_cross(self,censoring):
        def loss(y_true,y_pred):
  
            y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

            los = y_true * K.log(y_pred) 
            los = -K.mean(censoring*K.sum(los, -1))
            return los
        return loss
    
    #Custom keras constraint that clips small weights
    class Clipweights (Constraint):
        def __init__(self, clipval):
            self.clipval = clipval

        def __call__(self, w):
            new_w = w*K.cast(K.greater(K.abs(w),self.clipval),'float32')
            return new_w
        

    #Method that contains the model architecure and also builds the model. 
    #Requires regularisation constant lbda, the clipping threshold clpwt, optimizer opt,
    #loss_weights the relative importance of cox vs side channel and the model type 'elnet' 'lasso' or 'dropout'
    def build_model(self,lbda,clpwt,opt,lr,loss_weights,kind='elnet'):
        inputsx = Input(shape=(self.X.shape[1],))
        inputscancer = Input(shape=(self.cancer.shape[1],))
        ITM = Input(shape=(None,))
        state = Input(shape=(1,))
        Mc = Input(shape=(None,))
        Nc = Input(shape=(None,))
        Tc = Input(shape=(None,))

        
        if kind=='elnet':
            
            out = Dense(1,activation='linear',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13),
                       kernel_regularizer=l1_l2(lbda,lbda) ,kernel_constraint=self.Clipweights(0.001))(inputsx)
            
            outc = Dense(1, activation='linear',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13),#, use_bias=False,#kernel_initializer='zeros',
                      
                        name='cox')(out)

            outm = Dense(self.m.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#
                        )(out)
            outn = Dense(self.n.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#,
                        )(out)
            outt = Dense(self.t.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#,
                        )(out)
                        
            
        if kind=='linear_drop':
            out = Dropout(rate=lbda)(inputsx)
            out = Dense(1,activation='linear',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13),
                       kernel_constraint=self.Clipweights(0.001))(out)
            
            outc = Dense(1, activation='linear',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13),#, use_bias=False,#kernel_initializer='zeros',
                      
                        name='cox')(out)

            outm = Dense(self.m.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#
                        )(out)
            outn = Dense(self.n.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#,
                        )(out)
            outt = Dense(self.t.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#,
                        )(out)
            
        if kind=='linear_lasso':
            out = Dense(1,activation='linear',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13),
                       kernel_regularizer=l1_l2(lbda,0) ,kernel_constraint=self.Clipweights(0.001))(inputsx)
            
            outc = Dense(1, activation='linear',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13),#, use_bias=False,#kernel_initializer='zeros',
                      
                        name='cox')(out)

            outm = Dense(self.m.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#
                        )(out)
            outn = Dense(self.n.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#,
                        )(out)
            outt = Dense(self.t.shape[1], activation='softmax',kernel_initializer=VarianceScaling(scale=1,mode='fan_in', distribution='normal',seed=13)#, kernel_initializer='zeros'#,
                        )(out)
            

     
        
        model = Model(inputs=[inputsx, ITM, state,inputscancer,Mc,Nc,Tc], outputs=[outc,outm,outn,outt])
        model.compile(optimizer=opt ,
                      loss=[self.coxloss2(ITM,state),self.censored_cat_cross(Mc),
                            self.censored_cat_cross(Nc),self.censored_cat_cross(Tc)],loss_weights=loss_weights,experimental_run_tf_function=False)
        print(model.summary())
        self.model=model
        

        

        
        
    def train(self,iterations,batch):
        
        self.y_t2 = np.ones((len(self.X), 1))
        for a in range(iterations):
            ls_new = self.model.train_on_batch([self.X ,self.itm,self.state,self.cancer,self.mc,self.nc,self.tc], 
                                                            [np.ones((len(self.X), 1)),self.m,self.n,self.t])
            print(ls_new[0])
            self.tmplss.append(ls_new[0])
            if (self.tmplss[-1]>self.tmplss[-2])&(self.tmplss[-2]>self.tmplss[-3]):
                K.set_value(self.model.optimizer.lr,(K.get_value(self.model.optimizer.lr)/1.2))
                

        pr = self.predict()
        
        c = (concordance(self.state.astype('bool'), self.time, pr[0].flatten())) 
        
        return c
        
                      
    #method that runs a crossvalidation on all the data the class was initialised with.       
    def crossval(self, folds,lbda,clpwt,opt,loss_weights,kind,iterations,batch,stp,rseed=41):
        self.preds = pd.DataFrame(index=np.arange(len(self.X)))
        storeX = self.X
        storeitm = self.itm
        storestate= self.state
        storetime = self.time
        storecancer = self.cancer
        storem = self.m
        storemc = self.mc
        storen = self.n
        storenc = self.nc
        storet = self.t
        storetc = self.tc

        betas = []
        crossent = []
        
        cvconc_all = pd.DataFrame()
        nocvconc_all = pd.DataFrame()
        print(len(self.preds))
        
        kf = KF(n_splits = folds,shuffle=True,random_state=rseed)

        i=0
        for train_index,test_index in kf.split(self.X,self.state):
            cvtemp = []
            self.build_model(lbda,clpwt,opt,loss_weights,kind)
            K.set_value(self.model.optimizer.lr,0.001)
            self.tmplss = [1000,1000]
            cvconc = []
            nocvconc = []
            beta_temp = []
            for n in range(iterations//stp):
                print(K.get_value(self.model.optimizer.lr))
                self.X = storeX[train_index]
                self.state = storestate[train_index]
                self.itm = storeitm[train_index[:,None],train_index]
                self.time = storetime[train_index]
                self.cancer = storecancer[train_index]
                self.m = storem[train_index]
                self.mc = storemc[train_index]
                self.n = storen[train_index]
                self.nc = storenc[train_index]
                self.t = storet[train_index]
                self.tc = storetc[train_index]

                self.itshp=self.itm.shape 

                
                self.train(stp,batch)
                nocvconc.append(concordance(self.state.astype('bool'), self.time, self.predict()[0].flatten())[0])
                beta_temp.append(self.model.get_weights()[0])
                self.X = storeX[test_index]
                self.state = storestate[test_index]
                self.itm = storeitm[test_index[:,None],test_index]
                self.time = storetime[test_index]
                self.cancer = storecancer[test_index]
                self.m = storem[test_index]
                self.mc=storemc[test_index]
                self.n = storen[test_index]
                self.nc=storenc[test_index]
                self.t = storet[test_index]
                self.tc=storetc[test_index]


                self.preds = self.preds.merge(pd.DataFrame(self.predict()[0], index=test_index,columns=[str((n+1)*stp)]), left_index=True,right_index=True,how='outer')
                cvconc.append(concordance(self.state.astype('bool'), self.time, self.predict()[0].flatten())[0])
            i+=1
            betas.append(beta_temp)
            crossent.append( np.mean(np.sum(self.m* np.clip(self.predict()[1], K.epsilon(), 1 - K.epsilon()),-1  )))
            cvconc_all['fold'+str(i)] = cvconc
            nocvconc_all['fold'+str(i)] = nocvconc
        
        betas=np.squeeze(np.array(betas))
        print(betas.shape)
        
        sims = []
        nonzero = []
        size = []
        for a in range(betas.shape[1]):
            
            sims.append(np.mean(np.corrcoef(np.squeeze(betas[:,a,:]))))
            print('a')
            nonzero.append((np.count_nonzero(np.squeeze(betas[:,a,:])))/folds)
            print('b')
            size.append(np.mean(np.abs((np.squeeze(betas[:,a,:])))))
            print('c')
        
        self.preds = self.preds.fillna(0)   
        for z in range(stp,iterations+1,stp):
            sub = [mm for mm in list(self.preds) if str(z) in mm]
            self.preds['tot_'+str(z)] = self.preds[sub].sum(1)
        tots = [mm for mm in list(self.preds) if 'tot' in mm]
        print(len(self.preds))
        self.preds = self.preds[tots]
        print(len(self.preds))
        self.X = storeX
        self.itm = storeitm
        self.state = storestate
        self.time = storetime
        self.cancer = storecancer
        self.m = storem
        self.mc = storemc
        self.n = storen
        self.nc = storenc
        self.t = storet
        self.tc = storetc
        self.itshp=self.itm.shape

        return nocvconc_all, crossent,cvconc_all,sims,size,nonzero
            

            
    def predict(self):
        return self.model.predict([self.X, np.zeros(self.itshp), self.state,self.cancer,self.mc,self.nc,self.tc])

# Example loop run on the TCGA data across multiple cancers

In [262]:
#The list of cancers to be tested

cancerstotest = ['BRCA','PRAD','LUSC','CESC','KIRC','LIHC','COAD','HNSC','LUAD','LGG','PAAD','BLCA','KIRP','STAD']

In [ ]:
#This loop runs 3 crossvalidations for 3 different random seeds on each cancer listed 
#for a range of regularization parameters. 

%%time
results = pd.DataFrame(columns=['cancer','type','nodes','reg','weight','cv','var','nocv','nocv_var','sims','wtsize','nonzero','cv_max'])
scores=[]
for ctype in cancerstotest:
    

    plt.figure(figsize=(20,10))
    
   
    for f in ['elnet','linear_drop','linear_lasso']:
        if f=='elnet':
            regs = [0,0.0001,0.0003,0.001,0.003,0.01,0.03,0.1]
        if f=='linear_lasso':
            regs = [0,0.0002,0.0006,0.002,0.006,0.02,0.06,0.2]
        if f=='linear_drop':
            regs = [0,0.06125,0.125,0.25,0.5,0.75,0.885,0.93875]
        for b in regs:
            for d in [Adam(0.001)]:
                for e in [0,0.01,0.02,0.05,0.1,0.2,0.5,1]:
                    for rs in [42,43,44]:

                        K.clear_session()
                        try:
                            modlin = coxnetlin(X_exp[cancer[ctype]==1],y[cancer[ctype]==1]['vital_status'],y[cancer[ctype]==1]['time'],cancer[cancer[ctype]==1],itm.loc[cancer[cancer[ctype]==1].index,cancer[cancer[ctype]==1].index],M[cancer[ctype]==1],N[cancer[ctype]==1],T[cancer[ctype]==1])
                            s = modlin.crossval(5,b,[0,0],d,[1,e,e,e],f,250,0,50,rseed=rs)


                            scres  = pd.DataFrame(pd.DataFrame(s[2]).transpose().mean(),columns=['cv'])


                            scres['var'] = pd.DataFrame(s[2]).transpose().var()


                            scres['nocv'] = pd.DataFrame(s[0]).transpose().mean()
                            scres['nocv_var'] = pd.DataFrame(s[0]).transpose().var()
                            scres['sims'] = s[3]
                            scres['size'] = s[4]
                            scres['nonzero'] = s[5]
                            scores.append(scres)

                            print(K.get_value(modlin.model.optimizer.lr))


                            results.loc[len(results)] = [ctype,f,a,b,e]+list(np.array(scres)[-1])+[scres['cv'].max()]
                            print([ctype,f,a,b,e])


                            print(scres)

                        except:
                            results.loc[len(results)] = [ctype,f,a,b,e]+['fail','fail','fail','fail','fail','fail','fail','fail']
                            scres  = pd.DataFrame(pd.DataFrame([0]).transpose().mean(),columns=['cv'])

                            scres['var'] = [0]


                            scres['nocv'] = [0]
                            scres['nocv_var'] =[0]
                            scres['sims'] = [0]
                            scores.append(scres)
                            print('fail')
results.to_csv('crossval_results.csv')
